In [63]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [41]:
# Read csv
df_data = pd.read_csv('E:\OneDrive\Arbeit\Repos\DMC2022\Kevin\csv\complete_dataset_cleaned_labeled.csv', sep='|')
#df_data.head()

In [56]:
#df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1069416 entries, 0 to 1069415
Data columns (total 13 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   Unnamed: 0  1069416 non-null  int64 
 1   date        1069416 non-null  object
 2   userID      1069416 non-null  int64 
 3   itemID      1069416 non-null  int64 
 4   order       1069416 non-null  int64 
 5   brand       1069416 non-null  int64 
 6   feature_1   1069416 non-null  int64 
 7   feature_2   1069416 non-null  int64 
 8   feature_3   1069416 non-null  int64 
 9   feature_4   1069416 non-null  int64 
 10  feature_5   1069416 non-null  int64 
 11  categories  1069416 non-null  object
 12  week        1069416 non-null  int64 
dtypes: int64(11), object(2)
memory usage: 106.1+ MB


In [5]:
# Convert categories to data type category
#df_data["categories"] = df_data["categories"].astype("category")

In [6]:
#df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1069416 entries, 0 to 1069415
Data columns (total 13 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   Unnamed: 0  1069416 non-null  int64 
 1   date        1069416 non-null  object
 2   userID      1069416 non-null  int64 
 3   itemID      1069416 non-null  int64 
 4   order       1069416 non-null  int64 
 5   brand       1069416 non-null  int64 
 6   feature_1   1069416 non-null  int64 
 7   feature_2   1069416 non-null  int64 
 8   feature_3   1069416 non-null  int64 
 9   feature_4   1069416 non-null  int64 
 10  feature_5   1069416 non-null  int64 
 11  categories  1069416 non-null  object
 12  week        1069416 non-null  int64 
dtypes: int64(11), object(2)
memory usage: 106.1+ MB


# Preprocessing

In [42]:
# Drop 'Unnamed: 0' column
df_data.drop('Unnamed: 0', axis=1, inplace=True)

### Split datetime into separate columns

In [43]:
# Convert 'date' to datetime
df_data['date'] = pd.to_datetime(df_data['date'], format="%Y-%m-%d")

In [44]:
# Split 'date' into seperate columns
df_data['day'] = df_data['date'].dt.day
df_data['month'] = df_data['date'].dt.month
df_data['year'] = df_data['date'].dt.year

In [45]:
# Move new columns to the front
col = df_data.pop("year")
df_data.insert(1, col.name, col)

col = df_data.pop("month")
df_data.insert(2, col.name, col)

col = df_data.pop("day")
df_data.insert(3, col.name, col)

In [46]:
# Drop date column since it's no longer needed
df_data.drop('date', axis=1, inplace=True)

In [62]:
# Show table
df_data.head()

,year,month,day,userID,itemID,order,brand,feature_1,feature_2,feature_3,feature_4,feature_5,categories,week
0,2020,6,1,38769,3477,1,186,6,0,196,0,45,"[74, 4109, 3867, 803, 4053]",1
1,2020,6,1,42535,30474,1,193,10,3,229,3,132,"[3459, 3738, 679, 1628, 4072]",1
2,2020,6,1,42535,15833,1,1318,4,1,455,0,108,"[2973, 2907, 2749, 3357]",1
3,2020,6,1,42535,20131,1,347,4,0,291,3,44,"[30, 1515, 1760, 2932, 1287, 2615, 3727, 2450,...",1
4,2020,6,1,42535,4325,1,539,6,0,303,0,45,"[3104, 1772, 2029, 1274, 3915, 888, 1118, 3882...",1


### Multi-Hot-Encoding for categories

In [ ]:
"""
# If you run out of memory while encoding the whole dataframe at once you can 
# split the dataframe into chunks beforehand 

df_chunk1 = df.iloc[:500000,:]
df_chunk2 = df.iloc[500000:,:]

"""

In [8]:
# WARNING: This Cell takes a couple of minutes to compute, depending on how much RAM is available
# Convert categorical variable into dummy/indicator variables
df = df_data["categories"].str.get_dummies(',')

In [27]:
# Replace symbols in columns
df.columns = df.columns.str.replace("[", "")
df.columns = df.columns.str.replace("]", "")

C:\Users\kevin\AppData\Local\Temp\ipykernel_16228\3556933827.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.replace("[", "")
C:\Users\kevin\AppData\Local\Temp\ipykernel_16228\3556933827.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.replace("]", "")


In [26]:
df.head()

,1,1000,1000,1001,1002,1003,1003,1004,1006,1007,...,981,981,990,990,994,994,995,995,996,996
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
df_combined = df_data.join(df, how='outer')

In [30]:
df_combined.head()

,year,month,day,userID,itemID,order,brand,feature_1,feature_2,feature_3,...,981,981,990,990,994,994,995,995,996,996
0,2020,6,1,38769,3477,1,186,6,0,196,...,0,0,0,0,0,0,0,0,0,0
1,2020,6,1,42535,30474,1,193,10,3,229,...,0,0,0,0,0,0,0,0,0,0
2,2020,6,1,42535,15833,1,1318,4,1,455,...,0,0,0,0,0,0,0,0,0,0
3,2020,6,1,42535,20131,1,347,4,0,291,...,0,0,0,0,0,0,0,0,0,0
4,2020,6,1,42535,4325,1,539,6,0,303,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_combined_chunk1 = df.iloc[:500000,:]
df_combined_chunk2 = df.iloc[500000:,:]

In [33]:
col = df_combined.pop("week")
df_combined.insert(-1, col.name, col)

MemoryError: Unable to allocate 49.9 GiB for an array with shape (6269, 1069416) and data type int64

In [11]:
# Check if we have any missing values
df_combined[df_combined.isnull().any(axis=1)]

,1,1000,1000,1001,1002,1003,1003,1004,1006,1007,...,981,981,990,990,994,994,995,995,996,996


In [47]:
"""
For Testing purposes

# Read csv
df_cat = pd.read_csv('E:\OneDrive\Arbeit\Repos\DMC2022\Kevin\csv\complete_dataset_cleaned_labeled.csv', sep='|')
df_cat["categories"] = df_cat["categories"].astype("string")

df_cat.drop('Unnamed: 0', axis=1, inplace=True)
df_cat.drop('date', axis=1, inplace=True)
df_cat.drop('userID', axis=1, inplace=True)
df_cat.drop('itemID', axis=1, inplace=True)
df_cat.drop('order', axis=1, inplace=True)
df_cat.drop('brand', axis=1, inplace=True)
df_cat.drop('feature_1', axis=1, inplace=True)
df_cat.drop('feature_2', axis=1, inplace=True)
df_cat.drop('feature_3', axis=1, inplace=True)
df_cat.drop('feature_4', axis=1, inplace=True)
df_cat.drop('feature_5', axis=1, inplace=True)
df_cat.drop('week', axis=1, inplace=True)

df_cat = df_cat.head(3)
df_cat
"""

,categories
0,"[74, 4109, 3867, 803, 4053]"
1,"[3459, 3738, 679, 1628, 4072]"
2,"[2973, 2907, 2749, 3357]"


### Datatypes for XGBoost

XGBoost natively supports continuous data but not categorical data. In order to use categorical data with XGBoost, we have to use One-Hot-Encoding which converts a column of categorical values into muliple columns of binary values.

# Modeling

In [47]:
df_data.drop('categories', axis=1, inplace=True)

In [64]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1069416 entries, 0 to 1069415
Data columns (total 13 columns):
 #   Column     Non-Null Count    Dtype
---  ------     --------------    -----
 0   year       1069416 non-null  int64
 1   month      1069416 non-null  int64
 2   day        1069416 non-null  int64
 3   userID     1069416 non-null  int64
 4   itemID     1069416 non-null  int64
 5   order      1069416 non-null  int64
 6   brand      1069416 non-null  int64
 7   feature_1  1069416 non-null  int64
 8   feature_2  1069416 non-null  int64
 9   feature_3  1069416 non-null  int64
 10  feature_4  1069416 non-null  int64
 11  feature_5  1069416 non-null  int64
 12  week       1069416 non-null  int64
dtypes: int64(13)
memory usage: 106.1 MB


In [48]:
df_data.head()

,year,month,day,userID,itemID,order,brand,feature_1,feature_2,feature_3,feature_4,feature_5,week
0,2020,6,1,38769,3477,1,186,6,0,196,0,45,1
1,2020,6,1,42535,30474,1,193,10,3,229,3,132,1
2,2020,6,1,42535,15833,1,1318,4,1,455,0,108,1
3,2020,6,1,42535,20131,1,347,4,0,291,3,44,1
4,2020,6,1,42535,4325,1,539,6,0,303,0,45,1


In [62]:
# Comma is being used to extract a specific column from a 2D array.
# X = data.iloc[:,:-1]
# X = all rows, all columns except the last one 

X = df_data.iloc[:,0:-1]
X

,year,month,day,userID,itemID,order,brand,feature_1,feature_2,feature_3,feature_4,feature_5
0,2020,6,1,38769,3477,1,186,6,0,196,0,45
1,2020,6,1,42535,30474,1,193,10,3,229,3,132
2,2020,6,1,42535,15833,1,1318,4,1,455,0,108
3,2020,6,1,42535,20131,1,347,4,0,291,3,44
4,2020,6,1,42535,4325,1,539,6,0,303,0,45
...,...,...,...,...,...,...,...,...,...,...,...,...
1069411,2021,1,31,22114,29567,2,1445,3,0,-1,-1,-1
1069412,2021,1,31,22114,21068,1,193,4,0,453,3,108
1069413,2021,1,31,10900,18270,1,1383,6,0,537,0,46
1069414,2021,1,31,17894,31265,5,1137,4,0,398,0,144


In [50]:
y = df_data.iloc[:,-1]
y

0          1
1          1
2          1
3          1
4          1
          ..
1069411    4
1069412    4
1069413    4
1069414    4
1069415    4
Name: week, Length: 1069416, dtype: int64

In [51]:
# 
data_dmatrix = xgb.DMatrix(data=X, label=y)

C:\Users\kevin\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [52]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [58]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [59]:
xg_reg.fit(X_train,y_train)

preds = xg_reg.predict(X_test)

C:\Users\kevin\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[10:27:35] WARNING: d:\bld\xgboost-split_1645118015404\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.


In [60]:
# Je größer der RMSE ist, desto schlechter ist die Anpassung des Modells.
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 1.268038


In [56]:
model = XGBClassifier()
model.fit(X_train, y_train)

C:\Users\kevin\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\kevin\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[10:26:40] WARNING: D:\bld\xgboost-split_1645118015404\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=24,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [57]:
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 100.00%


In [ ]:
model = XGBClassifier(tree_method='gpu_hist', gpu_id=0)
model.fit(X_train, y_train)

In [61]:
!conda list

# packages in environment at C:\Users\kevin\anaconda3:
#
# Name                    Version                   Build  Channel
_ipyw_jlab_nb_ext_conf    0.1.0            py39haa95532_0  
_py-xgboost-mutex         2.0                       cpu_0    conda-forge
aiohttp                   3.8.1            py39h2bbff1b_1  
aiosignal                 1.2.0              pyhd3eb1b0_0  
alabaster                 0.7.12             pyhd3eb1b0_0  
anaconda                  2022.05                  py39_0  
anaconda-client           1.9.0            py39haa95532_0  
anaconda-navigator        2.2.0            py39haa95532_0  
anaconda-project          0.10.2             pyhd3eb1b0_0  
anyio                     3.5.0            py39haa95532_0  
appdirs                   1.4.4              pyhd3eb1b0_0  
argon2-cffi               21.3.0             pyhd3eb1b0_0  
argon2-cffi-bindings      21.2.0           py39h2bbff1b_0  
arrow                     1.2.2              pyhd3eb1b0_0  
astroid                

jupyter_client            6.1.12             pyhd3eb1b0_0  
jupyter_console           6.4.0              pyhd3eb1b0_0  
jupyter_core              4.9.2            py39haa95532_0  
jupyter_server            1.13.5             pyhd3eb1b0_0  
jupyterlab                3.3.2              pyhd3eb1b0_0  
jupyterlab_pygments       0.1.2                      py_0  
jupyterlab_server         2.10.3             pyhd3eb1b0_1  
jupyterlab_widgets        1.0.0              pyhd3eb1b0_1  
keyring                   23.4.0           py39haa95532_0  
kiwisolver                1.3.2            py39hd77b12b_0  
lazy-object-proxy         1.6.0            py39h2bbff1b_0  
lcms2                     2.12                 h83e58a3_0  
lerc                      3.0                  hd77b12b_0  
libaec                    1.0.4                h33f27b4_1  
libarchive                3.4.2                h5e25573_0  
libcrc32c                 1.1.1                ha925a31_2  
libcurl                   7.82.0        

win_inet_pton             1.1.0            py39haa95532_0  
win_unicode_console       0.5              py39haa95532_0  
wincertstore              0.2              py39haa95532_2  
winpty                    0.4.3                         4  
wrapt                     1.12.1           py39h196d8e1_1  
xarray                    0.20.1             pyhd3eb1b0_1  
xlrd                      2.0.1              pyhd3eb1b0_0  
xlsxwriter                3.0.3              pyhd3eb1b0_0  
xlwings                   0.24.9           py39haa95532_0  
xz                        5.2.5                h62dcd97_0  
yaml                      0.2.5                he774522_0  
yapf                      0.31.0             pyhd3eb1b0_0  
yarl                      1.6.3            py39h2bbff1b_0  
zfp                       0.5.5                hd77b12b_6  
zict                      2.0.0              pyhd3eb1b0_0  
zipp                      3.7.0              pyhd3eb1b0_0  
zlib                      1.2.12        